In [1]:
#importing all the libraries 
import numpy as np
import pandas as pd
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver import Chrome
import re
import csv
from getpass import getpass
from time import sleep
import getpass 

In [2]:
#twitter credentials 
twitterUsername=input("Enter Username:")
twitterPass=getpass.getpass("Enter password:")

Enter Username:abhinavchoudhary0000004@gmail.com
Enter password:········


In [10]:
#Enter all the search Query here
#Search Query used for this projects are mentioned in Last
#You can add multiple queries as well but its not recommended 
searchQuaries = [["Deloitte_Strategy&Transactions","Deloitte (Strategy OR Transactions) lang:en until:2020-11-29 since:2019-01-01 -filter:replies"]]

In [4]:
#collecting each tweet details 
def getTweetDetails(singleTweet):
    name = singleTweet.find_element_by_xpath('.//span').text
    try:
        twitterID = singleTweet.find_element_by_xpath('.//span[contains(text(), "@")]').text
    except NoSuchElementException:
        return
    try:
        date = singleTweet.find_element_by_xpath('.//time').get_attribute('datetime')
    except NoSuchElementException:
        return    
    comments = singleTweet.find_element_by_xpath('.//div[2]/div[2]/div[1]').text
    responding = singleTweet.find_element_by_xpath('.//div[2]/div[2]/div[2]').text
    text = comments + responding
    replyCount = singleTweet.find_element_by_xpath('.//div[@data-testid="reply"]').text
    retweetCount = singleTweet.find_element_by_xpath('.//div[@data-testid="retweet"]').text
    likeCount = singleTweet.find_element_by_xpath('.//div[@data-testid="like"]').text    
    tweetDetails = (name, twitterID, date, text, replyCount, retweetCount, likeCount)
    return tweetDetails    

In [5]:
#for Login into twitter account
driver = Chrome(executable_path=".\chromedriver.exe")
driver.get('https://www.twitter.com/login')
driver.maximize_window()
username = driver.find_element_by_xpath('//input[@name="session[username_or_email]"]')
username.send_keys(twitterUsername)
password = driver.find_element_by_xpath('//input[@name="session[password]"]')
password.send_keys(twitterPass)
password.send_keys(Keys.RETURN)
sleep(2)

In [11]:
#creating a iterative process to extract loops
for searchKey in searchQuaries:
    searchTextField = driver.find_element_by_xpath('//input[@aria-label="Search query"]')
    searchTextField.clear()
    searchTextField.send_keys(searchKey[1])
    searchTextField.send_keys(Keys.RETURN)
    driver.find_element_by_link_text('Latest').click()
    data = []
    tweetUnique = set()#to keep track of the tweets that has already been scrapped and to avoid scrapping same
    lastPos = driver.execute_script("return window.pageYOffset;")#keep track of scrolling position
    scroll = True
    maxtweets=0 #extract maximum of 1000 tweets
    while scroll:
        maxtweets=maxtweets+1
        pageTweets = driver.find_elements_by_xpath('//div[@data-testid="tweet"]')
        for singleTweet in pageTweets[-5:]: #to check if scrapped tweet is in last 5 tweets 
            tweet = getTweetDetails(singleTweet)
            if tweet:
                tweet_id = ''.join(tweet)
                if tweet_id not in tweetUnique:
                    tweetUnique.add(tweet_id)
                    data.append(tweet)
            
        attemp = 0 #for slow internet connectivity/reached end of Page
        while True:
            # check scroll position
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            sleep(2)
            currPos = driver.execute_script("return window.pageYOffset;")
            if lastPos == currPos:
                attemp = attemp + 1
                # end of scroll region
                if attemp >= 4:
                    scroll = False
                    break
                else:
                    sleep(5) # attempt another scroll
            else:
                lastPos = currPos
                if maxtweets>=500:
                    scroll = False
                break
    with open(searchKey[0]+'.csv', 'w', newline='', encoding='utf-8') as f:
        header = ['UserName', 'TwitterID', 'Time', 'Text','commentCount', 'retweetCount', 'likeCount']
        writer = csv.writer(f)
        writer.writerow(header)
        writer.writerows(data)
    #search_input.clear()
    driver.get("https://twitter.com/home")
    sleep(5)

In [ ]:
#for services
#EY (Strategy OR Transactions) lang:en until:2020-11-29 since:2019-01-01 -filter:replies
#EY Banking lang:en until:2020-11-29 since:2019-01-01 -filter:replies
#EY Tax lang:en until:2020-11-29 since:2019-01-01 -filter:replies
#EY Assurance lang:en until:2020-11-29 since:2019-01-01 -filter:replies
#EY Consultancy lang:en until:2020-11-29 since:2019-01-01 -filter:replies

#for grographical analysis
#(@EY_US) lang:en until:2020-11-29 since:2020-01-01 -filter:replies
#(@EY_India) lang:en until:2020-11-29 since:2020-01-01 -filter:replies
#(@EY_Australia) lang:en until:2020-11-29 since:2020-01-01 -filter:replies
#(@EY_UKI) lang:en until:2020-11-29 since:2020-01-01 -filter:replies
#(@EY_Africa) lang:en until:2020-11-29 since:2020-01-01 -filter:replies
#(@EYCanada) lang:en until:2020-11-29 since:2020-01-01 -filter:replies

#For Good and Bad
#(unhappy OR unsatisfied OR angry OR sad OR down OR disappointed OR miserable OR controversy OR sorry) (@EY OR @EY_India OR @EYnews OR @EY_EOY OR @EYPrivate OR @EYCanada OR @EY_PowerUtility OR @EY_Luxembourg OR @EY_TMT OR @EY_UKI OR @EY_people OR @EY_US OR @EY_Banking OR @EY_mena OR @EY_Tax OR @EY_Advisory OR @EYManufacturing OR @EY_Consulting OR @EY_Insurance) lang:en until:2020-11-29 since:2019-01-01 -filter:replies
#(happy OR excited OR amazing OR glad OR peaceful OR impressed OR positive OR convinced OR grateful OR satisfied) (@EY OR @EY_India OR @EYnews OR @EY_TMT OR @EY_UKI OR @EY_people OR @EY_US OR @EY_Banking OR @EY_MENa OR @EY_Tax OR @EY_Advisory OR @EYManufacturing OR @EY_Consulting OR @EY_Insurance) lang:en until:2020-11-29 since:2019-01-01 -filter:replies

#For Famous News Channel
#EY (from:CNBC OR from:CNNNews18 OR from:TimesNow OR from:ETNOWlive OR from:BNNBloomberg OR from:livemint OR from:EconomicTimes OR from:moneycontrolcom OR from:bsindia OR from:FinancialTimes) until:2020-11-30 since:2018-01-01 -filter:replies
#KPMG (from:CNBC OR from:CNNNews18 OR from:TimesNow OR from:ETNOWlive OR from:BNNBloomberg OR from:livemint OR from:EconomicTimes OR from:moneycontrolcom OR from:bsindia OR from:FinancialTimes) until:2020-11-30 since:2018-01-01 -filter:replies
#PWC (from:CNBC OR from:CNNNews18 OR from:TimesNow OR from:ETNOWlive OR from:BNNBloomberg OR from:livemint OR from:EconomicTimes OR from:moneycontrolcom OR from:bsindia OR from:FinancialTimes) until:2020-11-30 since:2018-01-01 -filter:replies
#Deloitte (from:CNBC OR from:CNNNews18 OR from:TimesNow OR from:ETNOWlive OR from:BNNBloomberg OR from:livemint OR from:EconomicTimes OR from:moneycontrolcom OR from:bsindia OR from:FinancialTimes) until:2020-11-30 since:2018-01-01 -filter:replies

#for Events and Technology
#(COVID OR corona) (@EY OR @EY_India OR @EYnews OR @EY_EOY OR @EYPrivate OR @EYCanada OR @EY_PowerUtility OR @EY_Luxembourg OR @EY_TMT OR @EY_UKI OR @EY_people OR @EY_US OR @EY_Banking OR @EY_mena OR @EY_Tax OR @EY_Advisory OR @EYManufacturing OR @EY_Consulting OR @EY_Insurance) lang:en until:2020-11-29 since:2020-01-01 -filter:replies
#("Artificial Intelligence" OR AI OR "Data Science" OR Blockchain OR "Machine Learning") (@EY OR @EY_India OR @EYnews OR @EY_EOY OR @EYPrivate OR @EYCanada OR @EY_PowerUtility OR @EY_Luxembourg OR @EY_TMT OR @EY_UKI OR @EY_people OR @EY_US OR @EY_Banking OR @EY_mena OR @EY_Tax OR @EY_Advisory OR @EYManufacturing OR @EY_Consulting OR @EY_Insurance) lang:en until:2020-11-29 since:2019-01-01 -filter:replies

In [ ]:
#for comparision

#geographical 
#(@PwC_IN) lang:en until:2020-11-29 since:2020-01-01 -filter:replies
#(@PwC_AU) lang:en until:2020-11-29 since:2020-01-01 -filter:replies
#(@PwC_UK) lang:en until:2020-11-29 since:2020-01-01 -filter:replies
#(@PwCUS) lang:en until:2020-11-29 since:2020-01-01 -filter:replies
#(@PwC_Canada) lang:en until:2020-11-29 since:2020-01-01 -filter:replies
#(@PwCAfrica) lang:en until:2020-11-29 since:2020-01-01 -filter:replies

#Similarly did all these for other two big4

In [ ]:
#official twitter accounts
#@PwC_IN OR @PwC OR @PwC_UK OR @PwCDigital OR @PwCUS OR @PwCAdvisory OR @PwCTax OR @PwCAssurance OR @PwCclimateready OR @pwc_ukgov OR @strategyand
#@KPMG OR @KPMGIndia OR @KPMG_US OR @KPMGUS_News OR @kpmguk OR @KPMG_China OR @KPMGUS_Tax OR @KPMGAUTaxNow OR @KPMGAdvisory
#@DeloitteIndia OR @DeloitteAcctg OR @DeloitteUS OR @lifeatdeloitte OR @DeloitteDigital OR @DeloitteOnTech OR @DeloitteFinSvc OR @DeloitteTMT OR DeloitteRiskFin OR @DeloitteTax OR @DeloitteStratOp

In [ ]:
#for sentiment analysis of other big 4
#(unhappy OR unsatisfied OR angry OR sad OR down OR disappointed OR miserable OR controversy OR sorry) (@PwC_IN OR @PwC OR @PwC_UK OR @PwCDigital OR @PwCUS OR @PwCAdvisory OR @PwCTax OR @PwCAssurance OR @PwCclimateready OR @pwc_ukgov OR @strategyand) lang:en until:2020-11-29 since:2019-01-01 -filter:replies
#(happy OR excited OR amazing OR glad OR peaceful OR impressed OR positive OR convinced OR grateful OR satisfied) (@PwC_IN OR @PwC OR @PwC_UK OR @PwCDigital OR @PwCUS OR @PwCAdvisory OR @PwCTax OR @PwCAssurance OR @PwCclimateready OR @pwc_ukgov OR @strategyand) lang:en until:2020-11-29 since:2019-01-01 -filter:replies
#(unhappy OR unsatisfied OR angry OR sad OR down OR disappointed OR miserable OR controversy OR sorry) (@KPMG OR @KPMGIndia OR @KPMG_US OR @KPMGUS_News OR @kpmguk OR @KPMG_China OR @KPMGUS_Tax OR @KPMGAUTaxNow OR @KPMGAdvisory) lang:en until:2020-11-29 since:2019-01-01 -filter:replies
#(happy OR excited OR amazing OR glad OR peaceful OR impressed OR positive OR convinced OR grateful OR satisfied) (@KPMG OR @KPMGIndia OR @KPMG_US OR @KPMGUS_News OR @kpmguk OR @KPMG_China OR @KPMGUS_Tax OR @KPMGAUTaxNow OR @KPMGAdvisory) lang:en until:2020-11-29 since:2019-01-01 -filter:replies
#(unhappy OR unsatisfied OR angry OR sad OR down OR disappointed OR miserable OR controversy OR sorry) (@DeloitteIndia OR @DeloitteAcctg OR @DeloitteUS OR @lifeatdeloitte OR @DeloitteDigital OR @DeloitteOnTech OR @DeloitteFinSvc OR @DeloitteTMT OR DeloitteRiskFin OR @DeloitteTax OR @DeloitteStratOp) lang:en until:2020-11-29 since:2019-01-01 -filter:replies
#(happy OR excited OR amazing OR glad OR peaceful OR impressed OR positive OR convinced OR grateful OR satisfied) (@DeloitteIndia OR @DeloitteAcctg OR @DeloitteUS OR @lifeatdeloitte OR @DeloitteDigital OR @DeloitteOnTech OR @DeloitteFinSvc OR @DeloitteTMT OR DeloitteRiskFin OR @DeloitteTax OR @DeloitteStratOp) lang:en until:2020-11-29 since:2019-01-01 -filter:replies


In [ ]:
#tweets with Big4 in it 
(#Big4) lang:en until:2020-11-30 since:2020-01-01 -filter:replies
    
#for services of Other Big4
#PWC (Strategy OR Transactions) lang:en until:2020-11-29 since:2019-01-01 -filter:replies
#PWC Banking lang:en until:2020-11-29 since:2019-01-01 -filter:replies
#PWC Tax lang:en until:2020-11-29 since:2019-01-01 -filter:replies
#PWC Assurance lang:en until:2020-11-29 since:2019-01-01 -filter:replies
#PWC Consultancy lang:en until:2020-11-29 since:2019-01-01 -filter:replies
    
#similarly done for all BIG4